In [1]:
from neuro_scraper import NeuroscienceNewsScraper

In [2]:
from joblib import Parallel, delayed

In [12]:
import functools
import operator

In [4]:
neuro_genres = NeuroscienceNewsScraper.neuro_genres

In [5]:
sc = NeuroscienceNewsScraper.NeuroscienceNewsSiteScraper()

In [6]:
urls = []

In [10]:
urls.extend(Parallel(n_jobs=-1)
           (delayed(sc.scrape_genre_for_article_urls) 
           (*neuro_genres[genre]) for genre in neuro_genres))

100%|██████████| 5/5 [00:00<00:00, 37.78it/s]


In [13]:
urls_flat = functools.reduce(operator.iconcat, urls, [])

In [19]:
urls_unique = list(set(urls_flat))

In [20]:
with open('neuroscience_news_urls.txt', 'w+') as text_file:
    for url in urls_unique:
        text_file.write("{url}\n".format(url=url))